In [1]:
%reset -f
import numpy as np
import tensorflow as tf
from numpy import array

In [2]:
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([25, 45, 65, 85, 105, 125, 145, 165, 185])

In [3]:
# stack columns
dataset = np.vstack((in_seq1, in_seq2, out_seq)).T
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [4]:
# splt a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X,y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the sequence
        if end_ix > len(sequence) -1:
            break
            
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix, :-1], sequence[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
        
    return array(X), array(y)

In [5]:
# choose a numbre of time steps
n_steps = 3

# split into sample
X, y  = split_sequence(dataset, n_steps)
print(X.shape, y.shape)

# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 2) (6,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165


In [6]:
# univariate lstm example
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

# define model
n_features = X.shape[2]
x0 = Input(shape = (n_steps, n_features))
x1 = LSTM(50, activation = 'relu')(x0)

output = Dense(1)(x1)

model = Model(inputs = x0, outputs = output)
model.compile(optimizer = 'adam', loss = 'mse')

# fit the model
model.fit(X, y, epochs = 200, verbose = 1)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 14008.9766
Epoch 2/200
1/1 [==============================] - 0s 5ms/step - loss: 13902.3018
Epoch 3/200
1/1 [==============================] - 0s 8ms/step - loss: 13798.5312
Epoch 4/200
1/1 [==============================] - 0s 5ms/step - loss: 13698.0898
Epoch 5/200
1/1 [==============================] - 0s 5ms/step - loss: 13601.3154
Epoch 6/200
1/1 [==============================] - 0s 7ms/step - loss: 13508.2695
Epoch 7/200
1/1 [==============================] - 0s 7ms/step - loss: 13419.4580
Epoch 8/200
1/1 [==============================] - 0s 9ms/step - loss: 13334.9541
Epoch 9/200
1/1 [==============================] - 0s 7ms/step - loss: 13253.3467
Epoch 10/200
1/1 [==============================] - 0s 9ms/step - loss: 13173.6924
Epoch 11/200
1/1 [==============================] - 0s 10ms/step - loss: 13095.0889
Epoch 12/200
1/1 [==============================] - 0s 6ms/step - loss: 13015.9834
Epoch 13/200


1/1 [==============================] - 0s 12ms/step - loss: 88.4816
Epoch 102/200
1/1 [==============================] - 0s 7ms/step - loss: 85.8310
Epoch 103/200
1/1 [==============================] - 0s 9ms/step - loss: 83.0218
Epoch 104/200
1/1 [==============================] - 0s 6ms/step - loss: 79.9953
Epoch 105/200
1/1 [==============================] - 0s 9ms/step - loss: 76.7200
Epoch 106/200
1/1 [==============================] - 0s 6ms/step - loss: 73.1819
Epoch 107/200
1/1 [==============================] - 0s 9ms/step - loss: 69.3757
Epoch 108/200
1/1 [==============================] - 0s 6ms/step - loss: 65.3101
Epoch 109/200
1/1 [==============================] - 0s 8ms/step - loss: 61.0255
Epoch 110/200
1/1 [==============================] - 0s 6ms/step - loss: 56.5963
Epoch 111/200
1/1 [==============================] - 0s 9ms/step - loss: 52.0910
Epoch 112/200
1/1 [==============================] - 0s 7ms/step - loss: 47.5318
Epoch 113/200
1/1 [======================

In [7]:
# demonstrate prediction
x_input = np.array([[80,85],[90,95],[100,105]])
x_input = x_input.reshape((1, n_steps, n_features))

y_pred = model.predict(x_input, verbose = 0)
print(y_pred)

[[207.53888]]


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3, 2)]            0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                10600     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 10,651
Trainable params: 10,651
Non-trainable params: 0
_________________________________________________________________
